In [2]:
# main.ipynb
import os
import time
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from data_processor import get_existing_ids
from data_processor import extract_and_save_data
from api_client import send_data_to_api

In [3]:
# Configuration du logger pour écrire dans un fichier log.txt
log_file = "log.txt"
logging.basicConfig(
    filename=log_file,  # Enregistre les logs dans ce fichier
    level=logging.INFO,  # Niveau de logging
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # Mode "append" pour conserver les anciens logs
)

In [8]:
# Paramètres
excel_file = "data.xlsx"
sheet_name = "Sheet1"

class ExcelFileHandler(FileSystemEventHandler):
    """ Classe pour gérer les événements de modification du fichier Excel """

    def on_modified(self, event):
        if event.src_path == os.path.abspath(excel_file):
            logging.info(f"Le fichier {excel_file} a été modifié, traitement en cours...")

            # Récupérer les ID existants via l'API
            existing_ids = get_existing_ids()

            # Extraire et filtrer les nouvelles données
            extract_and_save_data(excel_file, sheet_name, existing_ids)

            # Envoyer les données à l'API
            send_data_to_api()

# Démarrer l'observation du fichier
event_handler = ExcelFileHandler()
observer = Observer()
observer.schedule(event_handler, path=os.path.dirname(os.path.abspath(excel_file)), recursive=False)

logging.info(f"Surveillance du fichier {excel_file} en cours...")
observer.start()

logging.info("Le fichier Excel a été modifié, récupération des ID existants...")

# Récupérer les ID existants via l'API
existing_ids = get_existing_ids()  # 🔹 Appel de la fonction ici

logging.info("Extraction des nouvelles données en cours...")

# Extraire et filtrer les nouvelles données (en excluant celles déjà dans l'API)
extract_and_save_data(excel_file, sheet_name, existing_ids)  # 🔹 Passer `existing_ids`

logging.info("Données enregistrées dans pending_data.json.")

# Envoyer les données depuis le fichier JSON vers l'API
send_data_to_api()

try:
    while True:
        time.sleep(1)  # Garde le programme actif
except KeyboardInterrupt:
    logging.info("Arrêt du programme...")
    observer.stop()
    observer.join()

NameError: name 'get_existing_ids' is not defined